## How to use Anthropic Claude model
In this notebook we look into:
1. The basic on how to use an Anthropic model (Claude-3.5-Sonnet, and Claude-3.5-Haiku) with just a few lines of code
2. Which settings you can play with to tune the behaviour of the model on your use case.


**Table of content**
>[Anthropic Setup](#folderId=1tp_6Ep8ifTiLdokzSoJhJccd212GnCHV&updateTitle=true&scrollTo=trNIWx9Av671)

>[Simple inference with Claude model from Anthropic](#folderId=1tp_6Ep8ifTiLdokzSoJhJccd212GnCHV&updateTitle=true&scrollTo=UFa55z60wt72)

>[Advanced Options](#folderId=1tp_6Ep8ifTiLdokzSoJhJccd212GnCHV&updateTitle=true&scrollTo=sLX3RCdD1FHF)

>[Streaming](#folderId=1tp_6Ep8ifTiLdokzSoJhJccd212GnCHV&updateTitle=true&scrollTo=n2o2YPQl1t0s)



## Anthropic Setup

In [1]:
# First we have to install it (it i note available by default in Google Colab)
!pip install anthropic --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.8/288.8 kB 9.5 MB/s eta 0:00:00


In [2]:
import anthropic

In order to use an Anthropic Claude models, you'll need to create an API key and configure it in your Google Colab Secrets.


1. Login or create an Anthropic account, and configure a billing method, on the Anthropic Console [here](https://console.anthropic.com/login).
2. You'll also have to make provision a small amount of credit like $5 before you can start using the API.
3. Create a secret api key from [here](https://console.anthropic.com/settings/keys)
4. Open your Colab secrets (click on the key icon here on the left)
3. Give a the name, for instance `ANTHROPIC_API_KEY`, and past the value in `Value`.
4. Toggle `Notebook access` to give access to this specific notebook to the API key.

🔑 Note that this api key will now be available in your secrets everytime you open or create a new colab notebook. You'll however still need to grant explicit access to each notebook.


💸 Using an Anthropic model you will get charged! Use a small and cheap model for testing and learning like `claude-3-5-haiku-latest` then switch to a better model if needed for more complex tasks.


In [3]:
from google.colab import userdata
ANTHROPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')

## Simple inference with Claude model from Anthropic
Text generation is very simple. You need to create an Antrhopic `client` object. You then call the `client.messages.create()` function and pass **the two most important parameters:**
- 🧠 `model` the large language model being used.
- 💬 `messages` the list of user messages, and assistant responses.

It is very similar to OpenAI.

Note: However with Anthropic, the optional system prompt does not go into the messages list, instead, it is an extra argument **system** like illustrated below (more on system prompt with Claude [here](https://docs.anthropic.com/en/docs/build-with-claude/prompt-engineering/system-prompts).

### Anthropic Models
I recommend testing models in the following order (from cheaper to more expensive and better):
1. `claude-3-5-haiku-latest` affordable, intelligent and balzing fast.
    - Knowledge cut-off date: July 2024
2. `claude-sonnet-4-0` High intelligence and balanced performance.
    - Knowledge cut-off date: March 2025
3. `claude-opus-4-0`: Very expensive, but highest level of intelligence and capability
    - Knowledge cut-off date: March 2025

All models have a 200k tokens (150k english words) context window limits, i.e. the maximum size of input messages.

For pricing and more info, look [here](https://docs.anthropic.com/en/docs/about-claude/models/overview#model-comparison-table)

In [4]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

response = client.messages.create(
    model="claude-3-5-haiku-latest",
    max_tokens=2048,
    system="You are the World best poet", # <-- system prompt
    messages=[
        {"role": "user", "content": "Write a very short poem about an astronaut on the Moon"}
    ]
)

print(response.content[0].text)

Here's a short poem about an astronaut on the Moon:

Footprints in dust, silent and bright,
Alone on this pale, alien height.
Earth hangs distant, a blue-green sphere,
Silence echoes, no sound I hear.


## Advanced Options
Here are some more advanced parameters you can use.

In [5]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

response = client.messages.create(
    model="claude-3-5-haiku-latest",
    system="You are the World best poet",
    messages=[
        {"role": "user", "content": "Write a very short poem about an astronaut on the Moon"}
        ],
    max_tokens=512,
    temperature=0.8,
    top_k=10
)

In [6]:
print(response.content[0].text)

Here's a short poem about an astronaut on the Moon:

Footprints in dust, silent and bright,
Lone explorer in lunar light,
Earth hangs distant, blue and round,
Stillness echoes without a sound.


For the full documentation, check [here](https://docs.anthropic.com/en/api/messages). However, below are what I think the most important parameters to be aware of.

**Temperature**

- The `temperature` is the amount of randomness injected into the response. Defaults to 1.0. Ranges from 0.0 to 1.0. Use temperature closer to 0.0 for analytical / multiple choice, and closer to 1.0 for creative and generative tasks. Note that even with temperature of 0.0, the results will not be fully deterministic

**Maximum number of tokens**

- `max_token` refers to The maximum number of tokens to generate before stopping. Note that it may stop *before*.

**Top K**

- `top_k` top_k (integer) Only sample from the top K options for each subsequent token.

**System**

- `system` (string) a prompt you can use to set a role for the assistant.

# Streaming

Without streaming you have to wait until the full response is created by the model to see it.
With **streaming** you see each token as soos as they are generated, like in the Claude AI Chat interface. Streaming provide a much better user experience.
Otherwise, if you don't have user-facing apps, you may not need it.

In [7]:
import anthropic

client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)

with client.messages.stream(
    max_tokens=1024,
    messages=[{"role": "user", "content": "Write a very short poem about an astronaut on the Moon"}],
    model="claude-3-5-haiku-latest",
) as stream:
  for text in stream.text_stream:
      print(text, end="", flush=True)

Here's a short poem about an astronaut on the Moon:

Silent steps on silver ground,
Earth hangs distant, without a sound.
Footprints etched in lunar dust,
One small leap of human trust.